In [172]:
import pandas as pd
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings


pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [173]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [174]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [175]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
/02_regular_reports
astralab_banner


In [176]:
data_link = download_response.content

In [177]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [178]:
sheet_names.sheet_names

['Jul', 'Aug', 'Total']

In [179]:
report_type = 'test'

In [194]:
tmp_banner_dict = {}

sheet_names = pd.ExcelFile(BytesIO(data_link))
end_index = sheet_names.sheet_names.index('Total') # Забираем индекс листа, на котором находится агрегированная статистика

# проходим через цикл по списку названий листов БЕЗ учета листа Total
for name in sheet_names.sheet_names[:end_index]:
    cols_range = 'A:E' # задаем диапазон полей, которые нам нужны
    
    df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range)
    df = df.fillna(0)
    df = df.rename(columns={'Unnamed: 0': 'date', 'Clicks': 'clicks', 'Impressions': 'impressions', 'Reach': 'reach'})
    df = df[['date', 'reach', 'impressions', 'clicks']]
    
    product = 'main' # т.к. источник НЕ отдает название продукта - указываем для всех данных продукт main
    end_index = list(df[df['date']=='Total'].index)[0] # берем индекс окончания таблицы с данными
        
    df = df.iloc[:end_index] # оставляем строки с данными, которые нам нужны

    df['source'] = 'astralab' #добавляем название источника
    df['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
    df['date'] = pd.to_datetime(df['date']).dt.date  # приводим в формат даты

    df['product'] = product
    df['product'] = df['product'].str.lower().str.strip()
    df['report_type'] = report_type

    # сохраняем датаФрейм во временный словарь 
    # ключ - это название листа (Jul, Aug и тд)
    tmp_banner_dict[name] = df

In [195]:
tmp_banner_dict

{'Jul':           date  reach  impressions  clicks    source format_type product  \
 0   2024-07-17  29161        38944     256  astralab      banner    main   
 1   2024-07-18  22898        30524     174  astralab      banner    main   
 2   2024-07-19  14798        25821     180  astralab      banner    main   
 3   2024-07-20   8606        15980     123  astralab      banner    main   
 4   2024-07-21   7615        14960     126  astralab      banner    main   
 5   2024-07-22  12072        16045     101  astralab      banner    main   
 6   2024-07-23  16019        21819     173  astralab      banner    main   
 7   2024-07-24  17906        22678     172  astralab      banner    main   
 8   2024-07-25  18038        24713     138  astralab      banner    main   
 9   2024-07-26  12755        20124     178  astralab      banner    main   
 10  2024-07-27  14078        18258     181  astralab      banner    main   
 11  2024-07-28  11914        17797     322  astralab      banner    

In [187]:
cols_name = ['date', 'reach', 'impressions', 'views', 'clicks']

0            NaN
1         clicks
2    impressions
3            ctr
4          reach
Name: 0, dtype: object

In [193]:
df

,date,reach,impressions,clicks
0,2024-08-01 00:00:00,14481,17244,141
1,2024-08-02 00:00:00,21364,23637,134
2,2024-08-03 00:00:00,20647,23542,156
3,2024-08-04 00:00:00,19347,21726,156
4,2024-08-05 00:00:00,19086,21805,119
5,2024-08-06 00:00:00,19640,23331,146
6,2024-08-07 00:00:00,20763,23777,127
7,2024-08-08 00:00:00,21610,25186,163
8,2024-08-09 00:00:00,17055,18940,87
9,2024-08-10 00:00:00,17086,19035,84


In [ ]:
# источник Astralab
# тип отчета Баннерная реклама
# Функция для обработки статистики по дням для обычных размещений 
# Источник НЕ отдает название креативов, поэтому мы просто пишем main
# Статистика по каждому месяцу находится на отдельном листе

def get_astralab_banner_report(data_link, report_type):
    tmp_banner_dict = {}
    
    cols_range = 'A:E' # задаем диапазон полей, которые нам нужны
    # создаем список с названиями полей
    cols_name = ['date', 'reach', 'impressions', 'views', 'clicks']
    
    sheet_names = pd.ExcelFile(BytesIO(data_link))
    end_index = sheet_names.sheet_names.index('Total') # Забираем индекс листа, на котором находится агрегированная статистика
   
    # проходим через цикл по списку названий листов БЕЗ учета листа Total
    for name in sheet_names.sheet_names[:end_index]:
        cols_range = 'A:E' # задаем диапазон полей, которые нам нужны
        # создаем список с названиями полей
        cols_name = ['date', 'clicks', 'impressions', 'ctr', 'reach']
        
        df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range, names=cols_name)
        df = df[['date', 'clicks', 'impressions', 'reach']]
        df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить
        
        product_name = 'main' # т.к. источник НЕ отдает название продукта - указываем для всех данных продукт main
        end_index = list(df[df['date']=='Total'].index)[0] # берем индекс окончания таблицы с данными
        
        df = df.iloc[:end_index] # оставляем строки с данными, которые нам нужны
        df['source'] = 'astralab' #добавляем название источника
        df['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
        df['date'] = pd.to_datetime(df['date']).dt.date  # приводим в формат даты#df['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date().strftime('%Y-%m-%d'))#pd.to_datetime(df['date']).dt.date  # приводим в формат даты
        
        df.insert(1, 'product', product_name, True) # добавляем статичное поле с названием продукта
        df['product'] = df['product'].str.lower()
        df['report_type'] = report_type
        
        # сохраняем датаФрейм во временный словарь 
        # ключ - это название листа (Jul, Aug и тд)
        tmp_banner_dict[name] = df

    return pd.concat(tmp_banner_dict, ignore_index=True)